# Chapter 6: Hugging Face Integration

## Tapping into the World's AI Marketplace

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPOSITORY/blob/main/notebooks/06-HuggingFace-Integration.ipynb)

So far, we've mostly used proprietary models like Gemini as our agent's 'brain'. But what if you need a specialized model, or want to ground your agent in a specific dataset? Welcome to **Hugging Face**, the GitHub of machine learning. In this chapter, you'll learn how to integrate its vast resources into your agentic systems.

## 🤗 Why Integrate Hugging Face?

The Hugging Face Hub is a massive repository of:

- **Models:** Thousands of pre-trained models for various tasks (text generation, translation, summarization, etc.). You can find smaller, faster models or models with specializations.
- **Datasets:** Thousands of datasets you can use to give your agents specialized knowledge.
- **Tools:** A rich ecosystem of libraries like `transformers` and `datasets` that are the industry standard for working with ML models.

By integrating Hugging Face, you can build more powerful, flexible, and customized agents.

In [ ]:
# Step 1: Install and Setup
!pip install langchain transformers datasets huggingface_hub python-dotenv torch sentence-transformers

import os
from dotenv import load_dotenv
if not load_dotenv():
    try:
        from google.colab import userdata
        os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
        os.environ['HUGGINGFACE_API_KEY'] = userdata.get('HUGGINGFACE_API_KEY')
    except ImportError:
        print("Could not load API keys.")

### Part 1: Using a Hugging Face Model in LangChain

Let's swap out Gemini for an open-source model from the Hub. We'll use the `HuggingFacePipeline` class in LangChain, which makes it easy to work with `transformers` pipelines.

In [ ]:
# Step 2: Load a Hugging Face Model as the LLM
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Load a smaller, efficient model for demonstration
hf_llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="text2text-generation",
    pipeline_kwargs={"max_new_tokens": 100},
)

template = "Question: {question}\nAnswer:"
prompt = PromptTemplate.from_template(template)

chain = LLMChain(prompt=prompt, llm=hf_llm)

question = "What is the capital of France?"
print(chain.run(question))

### Part 2: Building a Hugging Face Dataset Tool

This is where it gets really powerful. We can create a tool that allows an agent to search for information within a specific Hugging Face dataset. This gives the agent specialized knowledge that the base LLM doesn't have.

In [ ]:
# Step 3: Define the Dataset Tool
from langchain.tools import tool
from datasets import load_dataset

@tool
def search_imdb_dataset(query: str) -> str:
    """Searches the IMDB dataset for movie reviews matching the query."""
    # Load the dataset (it will be downloaded and cached)
    imdb = load_dataset("imdb", split="train")
    # Perform a simple search
    results = imdb.filter(lambda example: query.lower() in example['text'].lower())
    
    if len(results) == 0:
        return f"No reviews found for '{query}'."
    
    # Return the first few results
    return "\n".join([res['text'][:200] + '...' for res in results[:3]])

print(search_imdb_dataset.name)
print(search_imdb_dataset.description)

In [ ]:
# Step 4: Create an Agent with the Dataset Tool
from langchain.agents import initialize_agent, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI

# Let's use Gemini as the 'brain' for its strong reasoning abilities
reasoning_llm = ChatGoogleGenerativeAI(model="gemini-pro")

tools = [search_imdb_dataset]
agent = initialize_agent(
    tools,
    reasoning_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Test the agent
agent.run("Find me a review of the movie 'Inception' that mentions the 'dream within a dream' concept.")

## ✅ Expanding the Possibilities

You've just given your agent a new superpower: the ability to query a massive, specialized dataset. This same pattern can be used to create all sorts of powerful tools:

- A **translation tool** that uses a `Helsinki-NLP` model.
- A **summarization tool** that uses a `facebook/bart-large-cnn` model.
- A **code generation tool** that uses a `CodeLlama` model.

By combining a powerful reasoning model (like Gemini) with specialized tool-using models and datasets from Hugging Face, you can build incredibly capable and knowledgeable agents.

In the next chapter, we'll go deeper into the **Gemini API** itself, exploring its native features like streaming and multimodal capabilities that go beyond the standard LangChain abstractions.